In [3]:
import torch
import numpy as np
import os
from torch import nn
import sys
import polars as pl
import sklearn
import shap
from functools import partial
import pandas as pd
import random
from matplotlib import pyplot as plt
import time
from rdkit.Chem import DataStructs
from tqdm import tqdm
from joblib import Parallel, delayed
import math
from skfp import fingerprints as skfps
from rdkit import Chem
from rdkit.Chem import AllChem

sys.path.append('../')
from src.utils.chem import normalize_smiles

In [4]:
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv').head(1000).select(
        pl.col('molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
print(train_df.estimated_size('gb'), 'GB')
train_df

7.411371916532516e-05 GB


molecule,bb1,bb2,bb3,BRD4,HSA,sEH
str,u16,u16,u16,u8,u8,u8
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0
…,…,…,…,…,…,…
"""C#CCOc1cccc(CN…",1640,1596,585,0,0,0
"""C#CCOc1cccc(CN…",1640,1596,1813,0,0,0
"""C#CCOc1cccc(CN…",1640,1596,1979,0,0,0


In [5]:
def create_3d_conformer_rdkit(mol, n = 1):
    new_mol = Chem.AddHs(mol)
    res = AllChem.EmbedMultipleConfs(new_mol, numConfs=n)
    ### MMFF generates multiple conformations
    res = AllChem.MMFFOptimizeMoleculeConfs(new_mol)
    new_mol = Chem.RemoveHs(new_mol)
    index = np.argmin([x[1] for x in res])
    energy = res[index][1]
    conf = new_mol.GetConformer(id=int(index))
    return conf

In [6]:
mols = [normalize_smiles(smiles,
                     canonical=True,
                     do_random = False,
                     isomeric=True,
                     kekulize = False,
                     replace_dy=True,
                     return_mol=True) for smiles in train_df[:, 'molecule']]
len(mols)

1000

In [19]:
%%time
for mol in tqdm(mols):
    xyz = create_3d_conformer_rdkit(mol).GetPositions()
    print(xyz.shape)

  0%|                                         | 1/1000 [00:00<11:38,  1.43it/s]

(41, 3)


  0%|                                         | 2/1000 [00:00<07:32,  2.20it/s]

(37, 3)


  0%|                                         | 3/1000 [00:01<08:32,  1.95it/s]

...

(40, 3)


100%|██████████████████████████████████████▉| 999/1000 [07:14<00:00,  2.86it/s]

(39, 3)


100%|██████████████████████████████████████| 1000/1000 [07:14<00:00,  2.30it/s]

(40, 3)
CPU times: user 7min 15s, sys: 515 ms, total: 7min 16s
Wall time: 7min 14s


In [20]:
from skfp.fingerprints import E3FPFingerprint
from skfp.preprocessing import MolFromSmilesTransformer, ConformerGenerator

In [21]:
%%time

conf_gen = ConformerGenerator()
mols = conf_gen.transform(mols)

CPU times: user 4min 13s, sys: 61.2 ms, total: 4min 14s
Wall time: 4min 14s


# Conforce

In [7]:
import sys
import CDPL.Chem as CDPLChem
import CDPL.ConfGen as ConfGen
import py3Dmol

# dictionary mapping status codes to human readable strings
STATUS_TO_STR = {
    ConfGen.ReturnCode.UNINITIALIZED: 'uninitialized',
    ConfGen.ReturnCode.TIMEOUT: 'max. processing time exceeded',
    ConfGen.ReturnCode.ABORTED: 'aborted',
    ConfGen.ReturnCode.FORCEFIELD_SETUP_FAILED: 'force field setup failed',
    ConfGen.ReturnCode.FORCEFIELD_MINIMIZATION_FAILED: 'force field structure refinement failed',
    ConfGen.ReturnCode.FRAGMENT_LIBRARY_NOT_SET: 'fragment library not available',
    ConfGen.ReturnCode.FRAGMENT_CONF_GEN_FAILED: 'fragment conformer generation failed',
    ConfGen.ReturnCode.FRAGMENT_CONF_GEN_TIMEOUT: 'fragment conformer generation timeout',
    ConfGen.ReturnCode.FRAGMENT_ALREADY_PROCESSED: 'fragment already processed',
    ConfGen.ReturnCode.TORSION_DRIVING_FAILED: 'torsion driving failed',
    ConfGen.ReturnCode.CONF_GEN_FAILED: 'conformer generation failed'
}


def gen3DStructure(smiles, struct_gen) -> int:
    smiles = normalize_smiles(
         smiles,
         canonical=True,
         do_random = False,
         isomeric=True,
         kekulize = False,
         replace_dy=True,
         return_mol=False)
    mol = CDPLChem.parseSMILES(smiles)
    # prepare the molecule for 3D structure generation
    ConfGen.prepareForConformerGeneration(mol)

    # generate the 3D structure
    status = struct_gen.generate(mol)

    # if successful, store the generated conformer ensemble as
    # per atom 3D coordinates arrays (= the way conformers are represented in CDPKit)

    if status == ConfGen.ReturnCode.SUCCESS:
        struct_gen.setCoordinates(mol)
        # enforce the output of 3D coordinates in case of MDL file formats
#         Chem.setMDLDimensionality(mol, 3)
    else:
        print(f'Error: 3D structure generation for molecule {smiles} failed: {STATUS_TO_STR[status]}')

    # return status code
    return status, mol

In [8]:
SDF_PATH = '/home/dangnh36/datasets/competitions/leash_belka/processed/features/3d_mols.sdf'

struct_gen = ConfGen.StructureGenerator()
struct_gen.settings.timeout = 3600 * 1000  # apply the -t argument

writer = CDPLChem.MolecularGraphWriter(SDF_PATH)

In [9]:
for i in tqdm(range(100)):
    smiles = train_df[i, 'molecule']
    status, mol = gen3DStructure(smiles, struct_gen)
    if not writer.write(mol):
        print(f'Error: writing 3D structure of molecule failed:\n{smiles}')
    # output the generated 3D structure
    # if not writer.write(mol):
    #     sys.exit('Error: writing 3D structure of molecule %s failed' % mol_id)

 31%|████████████▋                            | 31/100 [00:06<00:24,  2.81it/s]

Error: 3D structure generation for molecule C#CCOc1ccc(CNc2nc(Nc3ccc(B4OC(C)(C)C(C)(C)O4)cc3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1 failed: force field setup failed


100%|████████████████████████████████████████| 100/100 [00:16<00:00,  5.99it/s]


In [30]:
#visualization
rd_mols = Chem.SDMolSupplier(SDF_PATH, removeHs=False, sanitize=False)

p = py3Dmol.view(width=1200, height=400, viewergrid=(1,3))
#
for j,t in enumerate([10, 20, 30]):
    p.removeAllModels(viewer=(0,j))
    p.addModel(Chem.MolToMolBlock(rd_mols[t], confId=0), 'sdf', viewer=(0,j))
    p.setStyle({'stick':{}}, viewer=(0,j))
p.zoomTo()
p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [28]:
#visualization
rd_mols = Chem.SDMolSupplier(SDF_PATH)

p = py3Dmol.view(width=1200, height=400, viewergrid=(1,3))
#
for j,t in enumerate([10, 20, 30]):
    p.removeAllModels(viewer=(0,j))
    p.addModel(Chem.MolToMolBlock(rd_mols[t], confId=0), 'sdf', viewer=(0,j))
    p.setStyle({'stick':{}}, viewer=(0,j))
p.zoomTo()
p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [31]:
cdpl_mols = Chem.SDMolSupplier(SDF_PATH, removeHs=False, sanitize=False)


for i in range(1):
    cdpl_mol = cdpl_mols[i]
    cdpl_num_atoms = cdpl_mol.GetNumAtoms()
    cdpl_pos = cdpl_mols.GetItemText(i).split('\n')[4:4 + cdpl_num_atoms]
    cdpl_pos = np.array([[float(x) for x in line.split()[:3]] for line in cdpl_pos], dtype=np.float32)
    
    rd_mol = rd_mols[i]
    rd_pos = rd_mol.GetConformer().GetPositions()
#     assert sum(cdpl_pos - rd_pos) == 0
    print(cdpl_num_atoms, rd_mol.GetNumAtoms(), mols[i].GetNumAtoms())
    print(cdpl_pos, rd_pos, sep = '\n\n\n')

75 75 41
[[-9.34900e-01 -1.10000e-03  2.49600e+00]
 [-5.13700e-01  0.00000e+00  1.37200e+00]
 [ 0.00000e+00  0.00000e+00  0.00000e+00]
 [ 1.41800e+00 -0.00000e+00  0.00000e+00]
 [ 2.04770e+00 -0.00000e+00 -1.22340e+00]
 [ 1.30380e+00 -0.00000e+00 -2.40610e+00]
 [ 1.93850e+00  1.00000e-03 -3.65710e+00]
 [ 3.33660e+00 -9.00000e-04 -3.77890e+00]
 [ 4.01650e+00 -4.00000e-04 -5.10020e+00]
 [ 5.04440e+00 -1.00420e+00 -5.26320e+00]
 [ 4.81640e+00 -2.35020e+00 -5.56480e+00]
 [ 5.87160e+00 -3.17590e+00 -5.68810e+00]
 [ 5.58190e+00 -4.45180e+00 -5.98570e+00]
 [ 6.65200e+00 -5.33930e+00 -6.13230e+00]
 [ 6.53240e+00 -6.74480e+00 -6.45070e+00]
 [ 7.87980e+00 -7.41640e+00 -6.53600e+00]
 [ 8.65900e+00 -7.32510e+00 -5.21740e+00]
 [ 8.22960e+00 -8.57440e+00 -4.46200e+00]
 [ 8.11810e+00 -9.62030e+00 -5.56600e+00]
 [ 7.72080e+00 -8.85590e+00 -6.74100e+00]
 [ 7.41340e+00 -9.44280e+00 -7.97210e+00]
 [ 7.26510e+00 -1.08162e+01 -8.15150e+00]
 [ 6.96250e+00 -1.13663e+01 -9.40500e+00]
 [ 6.83310e+00 -1.04544e+

In [13]:
# kaggle_mol[0].GetConformer().GetPositions()